## Ejercicio 4: Mejorando la eficiencia con paralelismo
Investiga cómo asyncio puede integrarse con bibliotecas de procesamiento en paralelo como concurrent.futures para mejorar la eficiencia del servidor al manejar tareas que son intensivas en CPU.

Pasos:

- Modifica el servidor para utilizar concurrent.futures.ProcessPoolExecutor para ejecutar cálculos intensivos en paralelo.
- Crea tareas que requieran intensivo uso de CPU y envíalas al servidor.
- Observa y compara el rendimiento cuando se usan corutinas simples versus la ejecución en paralelo.

##cliente

In [ ]:
import dill as pickle
import socket
import random
from time import sleep

def submit_job(conn, data):
    conn.send(b'\x00')  
    data_bytes = pickle.dumps(data)
    conn.send(len(data_bytes).to_bytes(4, 'little'))
    conn.send(data_bytes)
    job_id = int.from_bytes(conn.recv(4), 'little')
    print(f'Enviada tarea al servidor. job_id: {job_id}')
    return job_id

def get_result(conn, job_id):
    conn.send(b'\x01')
    conn.send(job_id.to_bytes(4, 'little'))
    result_size = int.from_bytes(conn.recv(4), 'little')
    result = pickle.loads(conn.recv(result_size))
    print(f'El resultado es: {result}')

if __name__ == '__main__':
    conn = socket.create_connection(('127.0.0.1', 1936))
    
    job_ids = []
    for _ in range(10):
        numbers = [random.randint(1, 100) for _ in range(random.randint(10, 50))]
        job_id = submit_job(conn, numbers)
        job_ids.append(job_id)
    
    for job_id in job_ids:
        get_result(conn, job_id)
    
    conn.close()
